In [3]:
!wget --no-check-certificate \
    https://raw.githubusercontent.com/rishabhmisra/News-Headlines-Dataset-For-Sarcasm-Detection/master/Sarcasm_Headlines_Dataset.json \
    -O /tmp/Sarcasm_Headlines_Dataset.json


--2020-02-07 15:20:22--  https://raw.githubusercontent.com/rishabhmisra/News-Headlines-Dataset-For-Sarcasm-Detection/master/Sarcasm_Headlines_Dataset.json
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 64:ff9b::9765:85, 64:ff9b::9765:4085, 64:ff9b::9765:8085, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|64:ff9b::9765:85|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 6057046 (5.8M) [text/plain]
Saving to: ‘/tmp/Sarcasm_Headlines_Dataset.json’

/tmp/Sarcasm_Headli 100%[===================>]   5.78M  14.9MB/s    in 0.4s    

2020-02-07 15:20:23 (14.9 MB/s) - ‘/tmp/Sarcasm_Headlines_Dataset.json’ saved [6057046/6057046]



In [21]:
import json

datastore = []

with open('/tmp/Sarcasm_Headlines_Dataset.json') as f:
  for l in f:
    datastore.append(json.loads(l))
vocab_size = 4000
embedding_dim = 32
max_length = 120
trunc_type = 'post'
padding_type = 'post'
oov_token = '<OOV>'
training_size = 12000
num_epochs = 30


sentences = []
labels = []

for item in datastore:
  sentences.append(item['headline'])
  labels.append(item['is_sarcastic'])



In [22]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import numpy as np

training_sentences = sentences[0:training_size]
training_labels = np.array(labels[0:training_size])
testing_sentences = sentences[training_size:]
testing_labels = np.array(labels[training_size:])



tokenizer = Tokenizer(num_words=vocab_size, oov_token='<OOV>')
tokenizer.fit_on_texts(training_sentences)

# print(tokenizer.word_index)

training_sequences = tokenizer.texts_to_sequences(training_sentences)
training_padded = pad_sequences(training_sequences, 
                                maxlen=max_length, padding=padding_type,
                                truncating=trunc_type)

testing_sequences = tokenizer.texts_to_sequences(testing_sentences)
testing_padded = pad_sequences(testing_sequences, 
                                maxlen=max_length, padding=padding_type,
                                truncating=trunc_type)



In [26]:
model = tf.keras.Sequential([
  tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length=max_length),
  tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64, return_sequences=True)),
  tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(32)),
  tf.keras.layers.Dense(24, activation='relu'),
  tf.keras.layers.Dense(1, activation='sigmoid')
])


model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
model.summary()


fit_history = model.fit(training_padded, training_labels, 
                        epochs=num_epochs,
                        validation_data=(testing_padded, testing_labels), verbose=2)

import matplotlib.pyplot as plt

def plot_graphs(history, string):
  plt.plot(history.history[string])
  plt.plot(history.history['val_' + string])
  plt.xlabel("Epochs")
  plt.ylabel(string)
  plt.legend([string, 'val_' + string])
  plt.show()
  
plot_graphs(fit_history, "loss")
plot_graphs(fit_history, "accuracy")


Model: "sequential_9"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_11 (Embedding)     (None, 120, 32)           128000    
_________________________________________________________________
bidirectional_5 (Bidirection (None, 120, 128)          49664     
_________________________________________________________________
bidirectional_6 (Bidirection (None, 64)                41216     
_________________________________________________________________
dense_19 (Dense)             (None, 24)                1560      
_________________________________________________________________
dense_20 (Dense)             (None, 1)                 25        
Total params: 220,465
Trainable params: 220,465
Non-trainable params: 0
_________________________________________________________________
Train on 12000 samples, validate on 16619 samples
Epoch 1/30
12000/12000 - 43s - loss: 0.4515 - accuracy: 0.7763 -

KeyboardInterrupt: 